<a href="https://colab.research.google.com/github/KhanhPham2411/DeepReinforcementLearning/blob/master/funcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/KhanhPham2411/DeepReinforcementLearning/archive/master.zip

In [ ]:
!unzip master.zip

In [3]:
cd DeepReinforcementLearning-master/

/content/DeepReinforcementLearning-master


In [18]:
import numpy as np
import random

import loggers as lg

from game import Game, GameState
from model import Residual_CNN

from agent import Agent, User
from memory import Memory
import config

In [19]:
class DummyAgent():
	def __init__(self, name, state_size, action_size):
		self.name = name
		self.state_size = state_size
		self.action_size = action_size

	def act(self, state, tau):
		action = np.random.randint(0, self.action_size)
		pi = np.zeros(self.action_size)
		pi[action] = 1
		value = None
		NN_value = None
		return (action, pi, value, NN_value)

In [31]:
EPISODES = 5
logger = lg.logger_main
goes_first = 0
turns_until_tau0 = 10
MEMORY_SIZE = 30000

memory = Memory(MEMORY_SIZE)

env = Game()
player1 = DummyAgent('player1', env.state_size, env.action_size)
player2 = DummyAgent('player2', env.state_size, env.action_size)

scores = {player1.name:0, "drawn": 0, player2.name:0}
sp_scores = {'sp':0, "drawn": 0, 'nsp':0}
points = {player1.name:[], player2.name:[]}


In [ ]:
players[]

In [32]:
for e in range(EPISODES):

    logger.info('====================')
    logger.info('EPISODE %d OF %d', e+1, EPISODES)
    logger.info('====================')

    print (str(e+1) + ' ', end='')

    state = env.reset()
    
    done = 0
    turn = 0
    player1.mcts = None
    player2.mcts = None

    if goes_first == 0:
        player1Starts = random.randint(0,1) * 2 - 1
    else:
        player1Starts = goes_first

    if player1Starts == 1:
        players = {1:{"agent": player1, "name":player1.name}
                , -1: {"agent": player2, "name":player2.name}
                }
        logger.info(player1.name + ' plays as X')
    else:
        players = {1:{"agent": player2, "name":player2.name}
                , -1: {"agent": player1, "name":player1.name}
                }
        logger.info(player2.name + ' plays as X')
        logger.info('--------------')

    env.gameState.render(logger)

    print("EPISODES:" + str(e))

    while done == 0:
        turn = turn + 1
        

        #### Run the MCTS algo and return an action
        if turn < turns_until_tau0:
            action, pi, MCTS_value, NN_value = players[state.playerTurn]['agent'].act(state, 1)
        else:
            action, pi, MCTS_value, NN_value = players[state.playerTurn]['agent'].act(state, 0)

        if memory != None:
            ####Commit the move to memory
            memory.commit_stmemory(env.identities, state, pi)


        # logger.info('action: %d', action)
        # for r in range(env.grid_shape[0]):
        #     logger.info(['----' if x == 0 else '{0:.2f}'.format(np.round(x,2)) for x in pi[env.grid_shape[1]*r : (env.grid_shape[1]*r + env.grid_shape[1])]])
        # logger.info('MCTS perceived value for %s: %f', state.pieces[str(state.playerTurn)] ,np.round(MCTS_value,2))
        # logger.info('NN perceived value for %s: %f', state.pieces[str(state.playerTurn)] ,np.round(NN_value,2))
        # logger.info('====================')

        ### Do the action
        state, value, done, _ = env.step(action) #the value of the newState from the POV of the new playerTurn i.e. -1 if the previous player played a winning move
        
        env.gameState.render(logger)

        if done == 1: 
            if memory != None:
                #### If the game is finished, assign the values correctly to the game moves
                for move in memory.stmemory:
                    if move['playerTurn'] == state.playerTurn:
                        move['value'] = value
                    else:
                        move['value'] = -value
                        
                memory.commit_ltmemory()
            
            if value == 1:
                logger.info('%s WINS!', players[state.playerTurn]['name'])
                scores[players[state.playerTurn]['name']] = scores[players[state.playerTurn]['name']] + 1
                if state.playerTurn == 1: 
                    sp_scores['sp'] = sp_scores['sp'] + 1
                else:
                    sp_scores['nsp'] = sp_scores['nsp'] + 1

            elif value == -1:
                logger.info('%s WINS!', players[-state.playerTurn]['name'])
                scores[players[-state.playerTurn]['name']] = scores[players[-state.playerTurn]['name']] + 1
            
                if state.playerTurn == 1: 
                    sp_scores['nsp'] = sp_scores['nsp'] + 1
                else:
                    sp_scores['sp'] = sp_scores['sp'] + 1

            else:
                logger.info('DRAW...')
                scores['drawn'] = scores['drawn'] + 1
                sp_scores['drawn'] = sp_scores['drawn'] + 1

            pts = state.score
            points[players[state.playerTurn]['name']].append(pts[0])
            points[players[-state.playerTurn]['name']].append(pts[1])

print(scores)
print(memory)
print(points)
print(sp_scores)

1 EPISODES:0
2 EPISODES:1
3 EPISODES:2
4 EPISODES:3
5 EPISODES:4
{'player1': 3, 'drawn': 0, 'player2': 2}
{'player1': [1, 1, -1, 1, -1], 'player2': [-1, -1, 1, -1, 1]}
{'sp': 3, 'drawn': 0, 'nsp': 2}
